# Build Order

In [1]:
import qualified Data.Map as Map

In [2]:
dependencyGraph :: (Ord a) => [a] -> [(a, a)] -> Map.Map a [a]
dependencyGraph ps ds =
    Map.fromList [ (p, map snd $ filter ((p ==) . fst) ds) | p <- ps]

In [3]:
example = dependencyGraph
    ["a", "b", "c", "d", "e", "f"]
    [("d", "a"), ("b", "f"), ("d", "b"), ("a", "f"), ("c", "d")]
    
example

fromList [("a",["f"]),("b",["f"]),("c",["d"]),("d",["a","b"]),("e",[]),("f",[])]

In [4]:
import Data.Maybe
import Data.List

In [5]:
{-# LANGUAGE ScopedTypeVariables #-}

In [6]:
buildOrder :: forall a. (Ord a, Show a) => Map.Map a [a] -> Either String [a]
buildOrder deps =
    let
        computeState :: (Either String [a], [a]) -> a -> (Either String [a], [a])
        computeState (Left message, _) _ = (Left message, [])
        computeState (Right done, visited) key
            | key `elem` done = (Right done, visited)
            | key `elem` visited = (Left ("Cycle detected: " ++ show key), visited)
            | otherwise =
                let
                    tbd = filter (not.(`elem` done)) $ fromJust $ Map.lookup key deps -- !
                    (done', visited') = foldl' computeState (Right done, key:visited) tbd
                in
                    (fmap (++ [key]) done', visited')
    in
        fst $ foldl' computeState (Right [], []) $ Map.keys deps

buildOrder example
buildOrder $ Map.fromList [("a",["b"]),("b",["a"])]

Right ["f","a","b","d","c","e"]

Left "Cycle detected: \"a\""

In [7]:
import Test.QuickCheck

In [8]:
data DependencyGraph a = DependencyGraph (Map.Map a [a]) deriving (Show)

instance (Ord a, Arbitrary a) => Arbitrary (DependencyGraph a) where
    arbitrary = do
        keys <- fmap nub $ listOf arbitrary
        values <- sequence $ replicate (length keys) (sublistOf keys)
        return $ DependencyGraph (Map.fromList $ zip keys values)

In [9]:
testBasic :: DependencyGraph Int -> Bool
testBasic (DependencyGraph x) =
    let
        result = buildOrder x
    in
        case result of
            (Right order) -> Map.keys x == sort order
            (Left message) -> False

verboseCheckWith stdArgs { maxSize = 5 } testBasic

Passed:
DependencyGraph (fromList [])
Failed:
DependencyGraph (fromList [(-1,[-1])])
*** Failed! Falsifiable (after 2 tests): 
DependencyGraph (fromList [(-1,[-1])])